# **Feed Forward Neural Network** (FFNN) Implementation
#### _with **Forward Propagation**_

#### Made by:
- Samuel Christoper Swandi - 13520075
- Grace Claudia - 13520078
- Ubaidillah Ariq Prathama - 13520085
- Patrick Amadeus Irawan - 13520109

------------

List of Content

1. [Library & Dependencies](#library)
2. [Helper Function](#helper)
3. [Neural Network Visualization](#visualization)

## Library & Dependencies <a name="library"></a>

In [ ]:
!python3 -m pip install pyvis==0.3.2
!python3 -m pip install networkx==2.6.3
!python3 -m pip install numpy==1.21.6

In [1]:
import numpy as np
import os

from pyvis.network import Network

## Class

#### Connected Layer

In [2]:
class ConnectedLayer:
    def __init__(self, input_size, output_size, weights):
        self.input_size = input_size
        self.output_size = output_size
        self.weights = weights[1:]
        self.bias = weights[0]

    def forward(self, input):
        self.input = input
        return np.dot(input, self.weights) + self.bias

#### Activation Layer

In [3]:
class ActivationLayer:
    def __init__(self, activation):
        self.activation = activation
    
    def forward(self, input):
        self.input = input
        return self.activation(input)

#### Neural Network

In [198]:
class NeuralNetwork:
    def __init__(self):
        self.layers = []

    def add(self, layer):
        self.layers.append(layer)
    
    def predict(self, input_data):
        samples = len(input_data)
        complete_result = []
        result = []
        output = input_data[:, 1:]

        for layer in self.layers:
            output = layer.forward(output)
            if isinstance(layer, ActivationLayer):
                complete_result.append(output)

        return complete_result[-1] , complete_result

## **Helper** Function <a class="anchor" id="helper"></a>

#### Activation Function

In [226]:
def linear(net):
    return net

def ReLU(net):
    return np.maximum(0,net)

def sigmoid(net):
    return 1/(1+np.exp(-net))

def softmax(net):
    return np.exp(net)/np.sum(np.exp(net))

#### Loss Function

In [227]:
def mse(y_true, y_pred):
    return np.mean(np.square(y_true - y_pred))

def sse(y_true, y_pred):
    return np.sum(np.square(y_true - y_pred))

## Model Load Section

In [235]:
MODEL_FOLDER = '../test/model/'
INPUT_FOLDER = '../test/input/'
OUTPUT_FOLDER = '../test/output/'
# read file from test folder
def read_file(folder_path, file_name):
    with open(folder_path + file_name, 'r') as file:
        data = [i.rstrip("\n") for i in file.readlines()]
    return data

filename = input('Enter test case name (with extension): ')
data = read_file(MODEL_FOLDER, filename)
input_data = read_file(INPUT_FOLDER, filename)

try: 
    output_data = read_file(OUTPUT_FOLDER, filename)
except:
    output_data = None

## Loading Weights

In [236]:
idx = 1
weights = []
list_prev_size = []
list_layer_size = []
list_activation = []

for i in range(int(data[0]) - 1):
    prev_size, layer_size, activation = [int(i) for i in data[idx].split()]
    list_prev_size.append(prev_size)
    list_layer_size.append(layer_size)
    list_activation.append(activation)
    
    idx += 1
    weight = []
    for j in range(prev_size + 1):
        weight.append([float(i) for i in data[idx].split()])
        idx += 1
    
    weights.append(weight)

weights

[[[0.1, 0.2, 0.3], [0.4, -0.5, 0.6], [0.7, 0.8, -0.9]]]

## **Predict** Output

In [237]:
x_train = []
for i in range(len(input_data)):
    x_train.append([float(i) for i in input_data[i].split()])
x_train = np.array(x_train)
n = int(data[0])
act = {0: linear, 1: ReLU, 2: sigmoid, 3: softmax}

net = NeuralNetwork()

for i in range (n - 1):
    net.add(ConnectedLayer(list_prev_size[i], list_layer_size[i], weights[i]))
    net.add(ActivationLayer(act[list_activation[i]]))
    
out, complete_out = net.predict(x_train)

n_complete_out = []

for i in range(len(complete_out[0])):
    n_complete_out.append([complete_out[0][i], complete_out[-1][i]])

complete_out = n_complete_out

print("---PREDICTED OUTPUT---")
print(out)
print(complete_out)

---PREDICTED OUTPUT---
[[0.05 1.1  0.  ]]
[[array([0.05, 1.1 , 0.  ]), array([0.05, 1.1 , 0.  ])]]


## Compute Error with **Sum Squared Error (SSE)**

Sum Squared Error (SSE) is a mathematical function used in statistics and machine learning to measure the difference between predicted and actual values. It is commonly used as a cost function in various optimization algorithms, such as gradient descent.

The SSE is calculated by taking the difference between each predicted value and its corresponding actual value, squaring the difference, and then summing all of the squared differences:

$$SSE = \sum_{i=1}^{n}(y_i - \hat{y}_i)^2$$

Where:
- $n$ is the number of data points
- $y_i$ is the actual value of the i-th data point
- $\hat{y}_i$ is the predicted value of the i-th data point

The SSE gives an indication of how well the model fits the data. A lower SSE indicates that the model is a better fit for the data.

In machine learning, the SSE is often used as a cost function to be minimized during training of a model. The goal is to find the set of model parameters that minimizes the SSE, thus improving the accuracy of the model's predictions.

// create sse function with latex and give a brief explanation

In [238]:
parsed_output = [[float(j) for j in i.split()] for i in output_data]
out_pred = out.flatten()
out_true = np.array(parsed_output[:-1]).flatten()
max_sse = parsed_output[-1][0]

print('SSE  --->', sse(out_true, out_pred))
print("sse <= max_sse  --->", sse(out_true, out_pred) <= max_sse)

SSE  ---> 1.7333369499485123e-33
sse <= max_sse  ---> True


## Neural Network Visualization with **Pyvis**

Pyvis is a Python library that provides an easy-to-use interface for visualizing complex networks, including neural networks. Implementation of the visualization is enlisted below:

In [239]:
n = int(data[0])
act = {0: "linear", 1: "ReLU", 2: "sigmoid", 3: "softmax"}
idx = 1

XSTEP = 300
YSTEP = 300
SIZE = 10

nodes = []
node_i = 1
value = []
x_val = 0
y_val = 0
x = []
y = []
label = []
color = []
edge = []
title = []

src_idx = 0

for i_layer in range(n - 1):
    n_curr, n_next, act = [int(i) for i in data[idx].split()]

    if i_layer == 0:  # means that this is the first layer, hence construct input
        for i_node in range(n_curr + 1):
            if i_node == 0:  # bias
                color.append("#dd4b39")
                label.append("Input[bias]")
                temp = ""
                for i in range (len(input_data)):
                    temp += str(x_train[i][0])
                    if i < len(input_data) - 1:
                        temp += ", "
                title.append(temp)
            else:
                color.append("#162347")
                label.append("Input[{}]".format(i_node))
                temp = ""
                for i in range (len(input_data)):
                    temp += str(x_train[i][i_node])
                    if i < len(input_data) - 1:
                        temp += ", "
                title.append(temp)
            value.append(SIZE)
            x.append(x_val)
            y.append(y_val)
            y_val += YSTEP
            nodes.append(node_i)
            node_i += 1
        x_val += XSTEP

    y_val = 0
    # always construct the next layer
    for i_node in range(n_next + 1):
        if i_node == 0:
            if i_layer == n - 2:
                continue
            color.append("#dd4b39")
            label.append("HL{}[bias]".format(i_layer + 1))
            temp = ""
            for i in range (len(input_data)):
                temp +=  "1"
                if i < len(input_data) - 1:
                    temp += ", "
            title.append(temp)
        else:
            color.append("#162347")
            if i_layer == n - 2:
                label.append("Output[{}]".format(i_node))
            else:
                label.append("HL{}[{}]".format(i_layer + 1, i_node))
            temp = ""
            for i in range (len(input_data)):
                temp += str(complete_out[i][i_layer][i_node - 1])
                if i < len(input_data) - 1:
                    temp += ", "
            title.append(temp)
        value.append(SIZE)
        x.append(x_val)
        y.append(y_val)
        y_val += YSTEP
        nodes.append(node_i)
        node_i += 1
    x_val += XSTEP

    idx += 1
    for origin in range(n_curr + 1):
        dst_idx = -1
        for w in reversed(data[idx].split()):
            edge.append((nodes[src_idx], nodes[dst_idx], w))
            dst_idx -= 1
        src_idx += 1
        idx += 1


In [240]:
g = Network(notebook=True, cdn_resources="remote")
g.add_nodes(
	nodes,
    title = title,
    value = value,
    x=x,y=y,
    label = label,
    color = color
)

for e in edge:
    g.add_edge(e[0], e[1], title = e[2], color="#162347")

for n in g.nodes:
    n.update({'physics': False})
    
g.show("./tmp/network.html")

./tmp/network.html
